**Author**: Salma Elshahawy

**Date**: June, 18, 2020

**Title**: DATA 612, Recommender system, project#3

**Github repo**: [Matrix factorization using ALS project#3](https://github.com/salma71/MSDS_SU2020/blob/master/Recommender_system612/week_3/recommender_sys_databrick.ipynb)

## Introduction

For this week assignment, I used databricks notebook to build the recommender system, I used the movielens dataset with 20m observation

## Setting up databrick environment

In [4]:
# Instrument for unit tests. This is only executed in local unit tests, not in Databricks.
if 'dbutils' not in locals():
    import databricks_test
    databricks_test.inject_variables()

In [5]:
import os
# from test_helper import Test


dbfs_dir = '/databricks-datasets/cs110x/ml-20m/data-001'
ratings_filename = dbfs_dir + '/ratings.csv'
movies_filename = dbfs_dir + '/movies.csv'

# The following line is here to enable this notebook to be exported as source and
# run on a local machine with a local copy of the files. Just change the dbfs_dir,
# above.
if os.path.sep != '/':
  # Handle Windows.
  ratings_filename = ratings_filename.replace('/', os.path.sep)
  movie_filename = movie_filename.replace('/', os.path.sep)

In [6]:
display(dbutils.fs.ls(dbfs_dir))

path,name,size
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/README.txt,README.txt,8964
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/links.csv,links.csv,569517
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/links.csv.gz,links.csv.gz,245973
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/movies.csv,movies.csv,1397542
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/movies.csv.gz,movies.csv.gz,498839
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/ratings.csv,ratings.csv,533444411
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/ratings.csv.gz,ratings.csv.gz,132656084
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/tags.csv,tags.csv,16603996
dbfs:/databricks-datasets/cs110x/ml-20m/data-001/tags.csv.gz,tags.csv.gz,4787917


In [7]:
# deal with the comprissed files ending in (.gz)
from pyspark.sql.types import *

ratings_df_schema = StructType(
  [StructField('userId', IntegerType()),
   StructField('movieId', IntegerType()),
   StructField('rating', DoubleType())]
)
movies_df_schema = StructType(
  [StructField('ID', IntegerType()),
   StructField('title', StringType())]
)

In [8]:
# caching the data on top of S3 in memory 
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *

raw_ratings_df = sqlContext.read.format('com.databricks.spark.csv').options(header=True, inferSchema=False).schema(ratings_df_schema).load(ratings_filename)
ratings_df = raw_ratings_df.drop('Timestamp')

raw_movies_df = sqlContext.read.format('com.databricks.spark.csv').options(header=True, inferSchema=False).schema(movies_df_schema).load(movies_filename)
movies_df = raw_movies_df.drop('Genres').withColumnRenamed('movieId', 'ID')

ratings_df.cache()
movies_df.cache()

assert ratings_df.is_cached
assert movies_df.is_cached

raw_ratings_count = raw_ratings_df.count()
ratings_count = ratings_df.count()
raw_movies_count = raw_movies_df.count()
movies_count = movies_df.count()


## Reading the dataset

In [10]:
print ('There are %s ratings and %s movies in the datasets' % (ratings_count, movies_count))
print ('Ratings:')
ratings_df.show(3)
print ('Movies:')
display(movies_df.head(10))

assert raw_ratings_count == ratings_count
assert raw_movies_count == movies_count

ID,title
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
6,Heat (1995)
7,Sabrina (1995)
8,Tom and Huck (1995)
9,Sudden Death (1995)
10,GoldenEye (1995)


In [11]:
# quick verification on data
assert ratings_count == 20000263
assert movies_count == 27278
assert movies_df.filter(movies_df.title == 'Toy Story (1995)').count() == 1
assert ratings_df.filter((ratings_df.userId == 6) & (ratings_df.movieId == 1) & (ratings_df.rating == 5.0)).count() == 1


In [12]:
display(movies_df.head(10))

ID,title
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)
6,Heat (1995)
7,Sabrina (1995)
8,Tom and Huck (1995)
9,Sudden Death (1995)
10,GoldenEye (1995)


In [13]:
display(ratings_df.head(10))

userId,movieId,rating
1,2,3.5
1,29,3.5
1,32,3.5
1,47,3.5
1,50,3.5
1,112,3.5
1,151,4.0
1,223,4.0
1,253,4.0
1,260,4.0


One way to recommend movies is to find the movies with the highest average ratings.

I will use spark to find the name, number of ratings, and average ratings for the top 20 movies with at least 500 reviews.

* get ratings_df into a new dataframe called movie_ids_with_avg_ratings that have:
  1. movieId, 
  2. no_of ratings per movie, 
  3. avg rating for all movies

* add movie title column to the movie_ids_with_avg_ratings into a new dataframe called movie_names_with_avg_ratings_df
* will do join for the last step

In [15]:

from pyspark.sql import functions as F

movie_ids_with_avg_ratings_df = ratings_df.groupBy('movieId').agg(F.count(ratings_df.rating).alias('count'), F.avg(ratings_df.rating).alias('average'))

print('movie_ids_with_avg_ratings_df:')

display(movie_ids_with_avg_ratings_df.head(10))


movieId,count,average
3997,2047,2.0703468490473864
1580,35580,3.55831928049466
3918,1246,2.918940609951846
2366,6627,3.5492681454655197
3175,13945,3.600717102904267
4519,1936,3.2463842975206614
1591,5255,2.6201712654614653
471,11268,3.6641817536386228
36525,1169,3.482891360136869
44022,2465,3.334077079107505


In [16]:
movie_names_df = movies_df.withColumnRenamed('ID', 'movieId')
movie_names_with_avg_ratings_df = movie_ids_with_avg_ratings_df.join(movie_names_df, on=['movieId'], how='inner')
print('movie_names_with_avg_ratings_df:')
display(movie_names_with_avg_ratings_df.head(3))

movieId,count,average,title
3997,2047,2.0703468490473864,Dungeons & Dragons (2000)
1580,35580,3.55831928049466,Men in Black (a.k.a. MIB) (1997)
3918,1246,2.918940609951846,Hellbound: Hellraiser II (1988)


In [17]:
#Now that we have a DataFrame of the movies with highest average ratings, we can use Spark to determine the 20 movies with highest average ratings and at least 500 reviews. 

movies_with_500_ratings_or_more = movie_names_with_avg_ratings_df.filter(F.col('count')>=500)\
  .sort('average')

print ('Movies with highest ratings:')
display(movies_with_500_ratings_or_more.head(20))

movieId,count,average,title
4775,685,1.1240875912408759,Glitter (2001)
6587,701,1.1640513552068474,Gigli (2003)
1495,715,1.3902097902097903,Turbo: A Power Rangers Movie (1997)
50798,899,1.4888765294771968,Epic Movie (2007)
1739,623,1.5457463884430176,3 Ninjas: High Noon On Mega Mountain (1998)
4241,571,1.5726795096322241,Pokémon 3: The Movie (2001)
3593,3973,1.6005537377296752,Battlefield Earth (2000)
2386,591,1.619289340101523,Jerry Springer: Ringmaster (1998)
57532,615,1.6276422764227643,Meet the Spartans (2008)
1599,519,1.6666666666666667,Steel (1997)


## Creating a Training Set
break the ratings_df into three parts:

* A training set (DataFrame), which we will use to train models
* A validation set (DataFrame), which we will use to choose the best model
* A test set (DataFrame), which we will use for our experiments
* To randomly split the dataset into the multiple groups, we can use the pySpark randomSplit() transformation. randomSplit() takes a set of splits and a seed and returns multiple DataFrames.

In [19]:
# hold out 60% for training, 20% of our data for validation, and leave 20% for testing

seed = 42
(split_60_df, split_a_20_df, split_b_20_df) = ratings_df.randomSplit([0.6, 0.2, 0.2], seed = seed)

In [20]:
# cache the resulted datasets
training_df = split_60_df.cache()
validation_df = split_a_20_df.cache()
test_df = split_b_20_df.cache()

print('Training: {0}, validation: {1}, test: {2}\n'.format(
  training_df.count(), validation_df.count(), test_df.count())
)
display(training_df.head(3))
display(validation_df.head(3))
display(test_df.head(3))

userId,movieId,rating
1,29,3.5
1,32,3.5
1,47,3.5


After splitting the dataset, the training set has about 12 million entries and the validation and test sets each have about 4 million entries. (The exact number of entries in each dataset varies slightly due to the random nature of the randomSplit() transformation.)

## Alternating Least Squares
In this part, I will use the Apache Spark ML Pipeline implementation of Alternating Least Squares, ALS. 

ALS takes a training dataset (DataFrame) and several parameters that control the model creation process. To determine the best values for the parameters, I will use ALS to train several models, and then I will select the best model and use the parameters from that model in the rest.

The process to for determining the best model is as follows:

1. Pick a set of model parameters. The most important parameter to model is the **rank**, which is the number of columns in the Users matrix or the number of rows in the Movies matrix. A lower ran means higher error on the training dataset, but a high rank may lead to overfitting. I will train models with ranks of 4, 8, and 12 using the ```training_df``` dataset.

2. Set the appropriate parameters on the ALS object:

* The "User" column will be set to the values in the userId DataFrame column.
* The "Item" column will be set to the values in the movieId DataFrame column.
* The "Rating" column will be set to the values in the rating DataFrame column.
* I'll using a regularization parameter of 0.1.


3. Create multiple models using ALS.fit(), one for each of the rank values. I'll fit against the training data set ```(training_df)```.

For each model, I'll run a prediction against the validation data set (validation_df) and check the error.

I'll keep the model with the best error rate.

In [22]:
from pyspark.ml.recommendation import ALS

als = ALS(maxIter = 5, regParam = 0.1, userCol = 'userId', itemCol = 'movieId', ratingCol='rating', coldStartStrategy = 'drop')

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
# Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="rating", metricName="rmse")

tolerance = 0.03
ranks = [4, 8, 12]
errors = [0, 0, 0]
models = [0, 0, 0]
err = 0
min_error = float('inf')
best_rank = -1

for rank in ranks:
  # Set the rank here:
  als.setRank(5)
  # Create the model with these parameters.
  model = als.fit(training_df)
  # Run the model to create a prediction. Predict against the validation_df.
  predict_df = model.transform(validation_df)

  # Remove NaN values from prediction (due to SPARK-14489)
  predicted_ratings_df = predict_df.filter(predict_df.prediction != float('nan'))

  # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
  error = reg_eval.evaluate(predict_df)
  errors[err] = error
  models[err] = model
  print ('For rank %s the RMSE is %s' % (rank, error))
  if error < min_error:
    min_error = error
    best_rank = err
  err += 1

als.setRank(ranks[best_rank])
print ('The best model was trained with rank %s' % ranks[best_rank])
my_model = models[best_rank]

For rank 4 the RMSE is 0.8222388447502935
For rank 8 the RMSE is 0.8222388447502935
For rank 12 the RMSE is 0.8222388447502935
The best model was trained with rank 4

## Testing the Model
So far, I used the ```training_df``` and ```validation_df``` datasets to select the best model. Since I used these two datasets to determine what model is best, I cannot use them to test how good the model is; otherwise, I would be very vulnerable to **overfitting**. To decide how good the model is, I need to use the ```test_df``` dataset. I will use the ```best_rank``` I determined before to create a model for predicting the ratings for the test dataset and then we will compute the ```RMSE```.

The steps you should perform are:

* Run a prediction, using my_model as created above, on the test dataset (test_df), producing a new predict_df DataFrame.
* Filter out unwanted NaN values. We've supplied this piece of code for you.
* Use the previously created RMSE evaluator, reg_eval to evaluate the filtered DataFrame.

In [25]:
# In ML Pipelines, this next step has a bug that produces unwanted NaN values. We
# have to filter them out. See https://issues.apache.org/jira/browse/SPARK-14489
predict_df = my_model.transform(test_df)

# Remove NaN values from prediction (due to SPARK-14489)
predicted_test_df = predict_df.filter(predict_df.prediction != float('nan'))

# Run the previously created RMSE evaluator, reg_eval, on the predicted_test_df DataFrame
test_RMSE = reg_eval.evaluate(predicted_test_df)

print('The model had a RMSE on the test set of {0}'.format(test_RMSE))

The model had a RMSE on the test set of 0.8226318256461504

## Comparing the Model
Looking at the RMSE for the results predicted by the model versus the values in the test set is one way to evalute the quality of the model. Another way to evaluate the model is to evaluate the error from a test set where every rating is the average rating for the training set.

### The steps are:

1. Use the training_df to compute the average rating across all movies in that training dataset.
2. Use the average rating that you just determined and the test_df to create a DataFrame (test_for_avg_df) with a prediction column containing the average rating. 
3. Use the previously created reg_eval object to evaluate the test_for_avg_df and calculate the RMSE.

In [27]:
avg_rating_df = training_df.select(F.avg(training_df.rating))
avg_rating_df.show()

+------------------+
 avg(rating)|
+------------------+
3.5257348832825692|
+------------------+

In [28]:
# Compute the average rating

# Extract the average rating value. (This is row 0, column 0.)
training_avg_rating = avg_rating_df.collect()[0][0]
training_avg_rating
print('The average rating for movies in the training set is {0}'.format(training_avg_rating))

# Add a column with the average rating
# df.select(lit(5).alias('height')).withColumn('spark_user', lit(True)).take(1)
# [Row(height=5, spark_user=True)]
# test_for_avg_df = test_df.select(lit(training_avg_rating).alias('average')).withColumn('prediction', lit(True)).take(1)
test_for_avg_df = test_df.withColumn('prediction', F.lit(training_avg_rating))

# Run the previously created RMSE evaluator, reg_eval, on the test_for_avg_df DataFrame
test_avg_RMSE = reg_eval.evaluate(test_for_avg_df)

print("The RMSE on the average set is {0}".format(test_avg_RMSE))

The average rating for movies in the training set is 3.5257348832825692
The RMSE on the average set is 1.0521941515201385